# Simulações de rede QKD em topologia de malha

Este documento apresenta uma demonstração da criação e execução de requests em uma rede QKD com topologia de malha.

In [1]:
# Dependências
from components import Network, Controller
from components.qkd.app import *
import numpy as np

In [2]:
# Criando a rede
rede = Network()
controlador = Controller(rede)

In [3]:
# Gerando os requests
def generate_qkd_requests(rede, num_requests, apps, case):
        """
        Gera uma lista de requisições aleatórias de QKD.

        Args:
            num_requests (int): Número de requisições.
            diff_nodes (int): Número entre os nós. Defauts to 5.
            apps (list): Lista de apps disponíveis.
            
        Returns:
            requests (list): Lista com requisições.
        """
        classes = ["Class A", "Class B", "Class C", "Class D", "Class E"]
        requests = []
        
        if case == 0:
            class_distribution = [1/5] * 5
        elif case == 1:
            class_distribution = [0.2] * 5
        elif case == 2:
            class_distribution = [0.1, 0.1, 0.3, 0.25, 0.25]
        elif case == 3:
            class_distribution = [0.25, 0.25, 0.2, 0.1, 0.1]
        elif case == 4:
            class_distribution = [0.3, 0.3, 0.1, 0.15, 0.15]
        else:
            raise ValueError("Invalid case parameter")
            
        for i in range(num_requests):
            classe = random.choices(classes, class_distribution)[0]
            app = random.choice(apps)
            priority = random.randint(1, 5)
            alice, bob = rede.random_alice_bob()
            r = Request(classe, app, priority, alice, bob)
            requests.append(r)
        return requests

In [4]:
# Propriedades para simulação
n_simulacoes = 2
n_requests = 10
apps = ["BB84", "E91", "B92"]
caso = 0
rows = 3
cols = 4
topologia = "Lattice"

In [5]:
# Taxa de sucesso de bits das chaves
taxas_sucesso_chaves_geral = []

# Vazão da rede
vazao = []

In [6]:
# Executa num_simulacoes vezes simulações de requests
for indice, simulacao in enumerate(range(n_simulacoes)):
    
    # Variáveis para coleta dos dados
    taxas_sucesso_chaves_e91 = []
    taxas_sucesso_chaves_bb84 = []
    taxas_sucesso_chaves_b92 = []
    
    # Simulação
    rede.set_topology(topologia, rows, cols)
    requests = generate_qkd_requests(rede, n_requests, apps, caso)
    resultados_simulacao = controlador.send_requests(requests, 'kshortest')

    # Tratamento dos dados
    for indice_execucao in resultados_simulacao:

        # Percorre cada execução e colhe o sucesso nas chaves
        resultado_individual_simulacao = resultados_simulacao[indice_execucao]
        sucesso_chave = resultado_individual_simulacao['key sucess']

        # Colhendo o sucesso nas chaves de cada app (Tamamnho da chave recebida/ Tamanho da chave enviada)
        if resultado_individual_simulacao['app'] == 'BB84':
            taxas_sucesso_chaves_bb84.append(sucesso_chave)
            media_sucesso_chaves_bb84 = sum(taxas_sucesso_chaves_bb84) / len(taxas_sucesso_chaves_bb84)

        elif resultado_individual_simulacao['app'] == 'E91':
            taxas_sucesso_chaves_e91.append(sucesso_chave)
            media_sucesso_chaves_e91 = sum(taxas_sucesso_chaves_e91) / len(taxas_sucesso_chaves_e91)

        elif resultado_individual_simulacao['app'] == 'B92':
            taxas_sucesso_chaves_b92.append(sucesso_chave)
            media_sucesso_chaves_b92 = sum(taxas_sucesso_chaves_b92) / len(taxas_sucesso_chaves_b92)

    # Salvando o sucesso nas chaves geral da simulação
    lista_combinada = [taxa for sublist in [taxas_sucesso_chaves_bb84, taxas_sucesso_chaves_e91, taxas_sucesso_chaves_b92] for taxa in sublist]
    taxas_sucesso_chaves_geral.append(sum(lista_combinada) / len(lista_combinada))
    
    # Calculando a vazão
    n_execucoes = len(resultados_simulacao)
    vazao.append(n_requests / n_execucoes)

1ª EXECUÇÃO:
Requisições:  ['BB84: 10-8 (P:1 Key: 100)', 'E91: 2-8 (P:3 Key: 100)', 'E91: 3-9 (P:5 Key: 100)', 'BB84: 8-3 (P:2 Key: 100)', 'E91: 10-0 (P:4 Key: 100)', 'BB84: 2-0 (P:4 Key: 100)', 'BB84: 3-6 (P:2 Key: 100)', 'E91: 2-6 (P:3 Key: 100)', 'B92: 4-5 (P:3 Key: 100)', 'BB84: 7-6 (P:2 Key: 100)']
Requests ordenados por prioridade:  ['E91: 3-9 (P:5 Key: 100)', 'E91: 10-0 (P:4 Key: 100)', 'BB84: 2-0 (P:4 Key: 100)', 'E91: 2-8 (P:3 Key: 100)', 'E91: 2-6 (P:3 Key: 100)', 'B92: 4-5 (P:3 Key: 100)', 'BB84: 8-3 (P:2 Key: 100)', 'BB84: 3-6 (P:2 Key: 100)', 'BB84: 7-6 (P:2 Key: 100)', 'BB84: 10-8 (P:1 Key: 100)']
Rotas: [[3, 2, 1, 5, 9], [3, 7, 6, 5, 9], [3, 2, 6, 5, 9], [3, 7, 11, 10, 9], [3, 7, 6, 10, 9]]
Rota atual trabalhada:  [3, 2, 1, 5, 9]
Rotas: [[10, 6, 2, 1, 0], [10, 9, 8, 4, 0], [10, 6, 5, 1, 0], [10, 9, 5, 4, 0], [10, 6, 5, 4, 0]]
Rota atual trabalhada:  [10, 6, 2, 1, 0]
Rotas: [[2, 1, 0], [2, 6, 5, 4, 0], [2, 1, 5, 4, 0], [2, 6, 5, 1, 0], [2, 3, 7, 6, 5, 1, 0]]
Rota atual tr

## Resultados

In [7]:
print(np.mean(vazao))

2.916666666666667


In [8]:
print(np.mean(taxas_sucesso_chaves_geral))

0.46354166666666663


In [9]:
print(np.std(taxas_sucesso_chaves_geral))

0.03979166666666664
